In [1]:
import glob
import os
import re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# load data
txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

for txt_file in txt_list :
    with open(txt_file,'r') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print('data size :', len(raw_corpus))
print('Examples:\n', raw_corpus[:3])

data size : 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [3]:
# pre-processing data

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>' 
    if len(sentence.split()) >15 :
        sentence = ''
    return sentence

In [4]:
corpus =[]

for sentence in raw_corpus:
    if len(sentence)==0: continue
    if sentence[-1] ==':':continue
    if len(sentence.split())>= 12 : continue
        
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence) == 0: continue
    corpus.append(preprocessed_sentence)

In [5]:
len(corpus)

150057

In [6]:
# Tokenize

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,
        filters=' ',
        oov_token='<unk>')
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen=15,padding='post')
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [7]:
tensor , tokenizer = tokenize(corpus)

[[   2   51    5 ...    0    0    0]
 [   2   17 3146 ...    0    0    0]
 [   2   34    7 ...   43    3    0]
 ...
 [   2    5  120 ...    0    0    0]
 [   2  256  196 ...   12    3    0]
 [   2    7   35 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7ff684303040>


In [8]:
for idx in tokenizer.index_word:
    print(idx,":",tokenizer.index_word[idx])
    
    if idx >= 10 : break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [9]:
src_input = tensor[:,:-1]
tgt_input = tensor[:,1:]

print(src_input[0])
print(tgt_input[0])


[   2   51    5   91  317   62   55    9  938 6062    3    0    0    0]
[  51    5   91  317   62   55    9  938 6062    3    0    0    0    0]


In [10]:
# split train and test data
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,random_state=21,test_size=0.2)

In [11]:
# BUFFER_SIZE = len(src_input)
# BATCH_SIZE = 256
# steps_per_epoch = len(src_input) // BATCH_SIZE

# VOCAB_SIZE = tokenizer.num_words + 1

# dataset = tf.data.Dataset.from_tensor_slices((src_input,tgt_input))
# dataset = dataset.shuffle(BUFFER_SIZE)
# dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
# dataset

In [12]:
print('Source Train:',enc_train.shape)
print('Target Train:',dec_train.shape)

Source Train: (120045, 14)
Target Train: (120045, 14)


In [13]:
# model learning
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    


In [14]:
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train,dec_train,validation_data=(enc_val,dec_val),batch_size=256,epochs=10)

Epoch 1/10
469/469 [==============================] - 257s 503ms/step - loss: 3.2097 - val_loss: 2.8382
Epoch 2/10
469/469 [==============================] - 236s 503ms/step - loss: 2.6748 - val_loss: 2.5930
Epoch 3/10
469/469 [==============================] - 236s 503ms/step - loss: 2.3777 - val_loss: 2.4197
Epoch 4/10
469/469 [==============================] - 236s 504ms/step - loss: 2.0878 - val_loss: 2.2917
Epoch 5/10
469/469 [==============================] - 236s 504ms/step - loss: 1.8131 - val_loss: 2.1953
Epoch 6/10
469/469 [==============================] - 236s 504ms/step - loss: 1.5663 - val_loss: 2.1348
Epoch 7/10
469/469 [==============================] - 236s 504ms/step - loss: 1.3595 - val_loss: 2.0995
Epoch 8/10
469/469 [==============================] - 236s 504ms/step - loss: 1.1986 - val_loss: 2.0944
Epoch 9/10
469/469 [==============================] - 236s 504ms/step - loss: 1.0889 - val_loss: 2.1070
Epoch 10/10
469/469 [==============================] - 236s 504m

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
   
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
   
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i hate", max_len=20)

'<start> i hate a goofy especially <end> '